# PageRank Algorithm Implementation and Exercises

In this notebook we study and implement the PageRank algorithm following the formulation described by Bryan and Leise in **“The $25{,}000{,}000{,}000$ Eigenvector: The Linear Algebra Behind Google”**.  
The objective is to provide a mathematically sound implementation of the algorithm, validate it on the example graphs presented in the paper, and solve selected exercises.

We construct small link matrices corresponding to Figures 2.1 and 2.2, implement the PageRank computation using the power method, and solve Exercises 11 and 14 from the paper. An additional cell allows the algorithm to be tested on a real dataset (e.g. the Hollins University web graph), provided that a file named `hollins.dat` is available in the working directory.

The PageRank algorithm computes the stationary distribution of a stochastic matrix
$$
M = (1 - m)\,A + m\,S
$$
where $(A) $ is a column-stochastic link matrix representing the web graph, $(S) $ is the teleportation matrix with all entries equal to \(1/n\), and $(m \in (0,1)) $$ is the damping factor (typically $ (m = 0.15)$).  
The PageRank vector is defined as the dominant eigenvector of $(M) $ associated with the eigenvalue 1, and it is computed numerically using the power method.


## Construction of the Link Matrices and PageRank Implementation

In this section we define the link matrices corresponding to the example graphs presented in the paper and implement the PageRank algorithm using the power method.

### Link Matrices for the Example Graphs

For the small examples in Figures 2.1 and 2.2, the web is represented by a directed graph and encoded in a matrix $$(A \in \mathbb{R}^{n \times n})$$ .  
The convention used throughout this notebook is the following:

- Each column $(j)$ represents page $(j)$ .
- The nonzero entries in column $(j)$ indicate the pages reached by outgoing links from page $(j)$.
- If page $(j)$ has $(d_j)$ outgoing links, each destination receives a weight $(1/d_j)$.

Using this convention, the matrices for Figures 2.1 and 2.2 are constructed explicitly by assigning the appropriate nonzero entries.  
Exercise 11 is handled by extending the matrix of Figure 2.1 with an additional page and modifying the corresponding links as described in the exercise.

### Handling of Dangling Nodes

A page with no outgoing links corresponds to a zero column in the matrix $(A)$.  
Such pages are called *dangling nodes* and would break the stochasticity of the matrix. To handle this case, any zero column is replaced by a uniform distribution over all pages. This ensures that the resulting matrix is column-stochastic.

### PageRank Computation via the Power Method

Given a column-stochastic matrix $(A)$, the PageRank algorithm is based on the Google matrix
$$
M = (1 - m)A + mS
$$
where $(m)$ is the damping factor and $(S)$ is the matrix whose entries are all equal to $(1/n)$.

The PageRank vector is defined as the dominant eigenvector of $(M)$ associated with the eigenvalue 1.  
It is computed numerically using the power method, starting from a uniform probability vector. At each iteration, the vector is normalized to avoid numerical drift, and convergence is checked using the $( \ell^1 )$-norm of the difference between successive iterates.


In [1]:
import numpy as np
from typing import Tuple

def build_matrix_fig21() -> np.ndarray:
    """Return the 4×4 link matrix for Figure 2.1: each column j lists the destinations of page j."""
    A = np.zeros((4, 4), dtype=float)
    # Page 1 links to 2,3,4 with equal weight
    A[1, 0] = A[2, 0] = A[3, 0] = 1/3
    # Page 2 links to 3,4
    A[2, 1] = A[3, 1] = 1/2
    # Page 3 links only to 1
    A[0, 2] = 1.0
    # Page 4 links to 1 and 3
    A[0, 3] = A[2, 3] = 1/2
    return A

def build_matrix_fig22() -> np.ndarray:
    """Return the 5×5 link matrix for Figure 2.2 (two disconnected subwebs)."""
    A = np.zeros((5, 5), dtype=float)
    # Subweb 1: pages 1↔2
    A[1, 0] = 1.0
    A[0, 1] = 1.0
    # Subweb 2: pages 3↔4
    A[3, 2] = 1.0
    A[2, 3] = 1.0
    # Page 5 links to 3 and 4 with equal weight
    A[2, 4] = A[3, 4] = 0.5
    return A

def build_matrix_exercise11() -> np.ndarray:
    """Return the 5×5 link matrix for Exercise 11 (Figure 2.1 + a fifth page)."""
    A = np.zeros((5, 5), dtype=float)
    # Original four pages
    A[1, 0] = A[2, 0] = A[3, 0] = 1/3
    A[2, 1] = A[3, 1] = 1/2
    # Page 3 now links to pages 1 and 5
    A[0, 2] = A[4, 2] = 1/2
    # Page 4 links to pages 1 and 3
    A[0, 3] = A[2, 3] = 1/2
    # New page 5 links only to page 3
    A[2, 4] = 1.0
    return A

def ensure_column_stochastic(A: np.ndarray) -> np.ndarray:
    """Return a column‑stochastic version of A: normalizes nonzero columns, replaces zero-columns with uniform."""
    n = A.shape[0]
    B = np.zeros_like(A, dtype=float)
    for j in range(n):
        col = A[:, j]
        s = col.sum()
        if s > 0:
            B[:, j] = col / s
        else:
            B[:, j] = 1.0 / n  # dangling node: distribute uniformly
    return B

def pagerank(A: np.ndarray, m: float = 0.15, tol: float = 1e-12, max_iter: int = 10000) -> Tuple[np.ndarray, int]:
    """
    Compute the PageRank vector for the column‑stochastic matrix A using the power method.

    Parameters
    ----------
    A : np.ndarray
        Column‑stochastic link matrix.
    m : float
        Damping factor (default 0.15).
    tol : float
        Convergence tolerance in L1 norm (default 1e-12).
    max_iter : int
        Maximum number of power iterations (default 10,000).

    Returns
    -------
    v : np.ndarray
        PageRank vector.
    k : int
        Number of iterations used.
    """
    # Ensure A is column‑stochastic for safety
    A = ensure_column_stochastic(A)
    n = A.shape[0]
    # Google matrix
    S = np.ones((n, n), dtype=float) / n
    M = (1 - m) * A + m * S
    # Start from uniform distribution
    v = np.ones(n, dtype=float) / n
    for k in range(1, max_iter + 1):
        v_next = M @ v
        v_next = v_next / v_next.sum()  # normalize to avoid drift
        if np.abs(v_next - v).sum() < tol:
            return v_next, k
        v = v_next
    return v, max_iter


## PageRank Computation on a Real Dataset

To apply the PageRank algorithm to a real web graph, the dataset is provided as an edge list and converted into an adjacency-list representation. Each line of the file represents a directed link between two nodes.

### Loading the Dataset

The dataset is parsed and stored as a dictionary mapping each node to the list of its outgoing neighbors.  
During this process:
- Commented or malformed lines are ignored.
- Self-loops can be optionally removed.
- The direction of edges can be flipped if needed, depending on the dataset convention.

This representation is suitable for efficient iteration over outgoing links and scales well to large graphs.

### PageRank Power Method on Adjacency Lists

For large datasets, explicitly storing the matrix $(A)$ is inefficient. Instead, the PageRank iteration is implemented directly on the adjacency list.

Given the current PageRank vector $(v) $, the update step is:
$$
v^{(k+1)} = (1 - m)\,\hat{A}v^{(k)} + \frac{m}{n}\mathbf{1}
$$
where \(\hat{A}\) denotes the implicit column-stochastic link structure.

Pages with no outgoing links (dangling nodes) are handled by accumulating their PageRank mass and redistributing it uniformly across all nodes at each iteration. This preserves stochasticity and guarantees convergence.

At each iteration:
- The vector is normalized to avoid numerical drift.
- Convergence is checked using the \( \ell^1 \)-norm:
$$
\|v^{(k+1)} - v^{(k)}\|_1 < \text{tol}
$$

The algorithm stops once the tolerance is met or the maximum number of iterations is reached.

### Dataset Availability

The dataset is loaded only if a file named `hollins.dat` is found in the working directory. If the file is not present, the dataset analysis is skipped without affecting the rest of the notebook.


In [2]:
import os
import numpy as np
from typing import Dict, List, Tuple

def load_edge_list_as_adj(path: str, *, flip_direction: bool = False, drop_self_loops: bool = True) -> Dict[int, List[int]]:
    edges: List[Tuple[int, int]] = []
    nodes: set[int] = set()
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith('#') or line.startswith('//') or line.startswith('%'):
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            try:
                a = int(parts[0]); b = int(parts[1])
            except ValueError:
                continue
            u, v = (b, a) if flip_direction else (a, b)
            if drop_self_loops and u == v:
                continue
            edges.append((u, v))
            nodes.update({u, v})
    adj: Dict[int, List[int]] = {node: [] for node in sorted(nodes)}
    for u, v in edges:
        adj[u].append(v)
    return adj

def pagerank_power(adj: Dict[int, List[int]], m: float = 0.15, tol: float = 1e-12, max_iter: int = 1000) -> Tuple[Dict[int, float], int, float]:
    nodes = list(adj.keys())
    n = len(nodes)
    idx = {node: i for i, node in enumerate(nodes)}
    outdeg = [len(adj[node]) for node in nodes]

    v = np.ones(n) / n
    teleport = m / n
    one_minus_m = 1.0 - m

    last_diff = float("inf")
    it = 0

    for it in range(1, max_iter + 1):
        v_next = np.zeros(n)
        dangling_mass = 0.0

        for j, node in enumerate(nodes):
            if outdeg[j] == 0:
                dangling_mass += v[j]
            else:
                share = v[j] / outdeg[j]
                for dest in adj[node]:
                    v_next[idx[dest]] += share

        if dangling_mass > 0:
            v_next += (dangling_mass / n)

        v_next = one_minus_m * v_next + teleport

        # Normalize at each iteration to avoid numerical drift
        s = v_next.sum()
        if s != 0:
            v_next /= s

        last_diff = np.abs(v_next - v).sum()

        v = v_next
        if last_diff < tol:
            break

    scores = {node: float(v[idx[node]]) for node in nodes}
    return scores, it, last_diff


candidates = ['/mnt/data/hollins.dat', 'hollins.dat']
file_path = next((p for p in candidates if os.path.exists(p)), None)
print("file_path =", file_path)


file_path = hollins.dat


## Ranking with Ties

In some cases, different nodes may obtain identical or nearly identical PageRank values due to symmetry in the link structure or numerical precision limits. A simple sorting of the scores would impose an artificial ordering among such nodes.

To address this issue, a tie-aware ranking procedure is introduced. Nodes are first sorted by decreasing PageRank score, and then grouped into the same rank whenever the absolute difference between their scores is below a prescribed tolerance $( \varepsilon ) $.

The ranking scheme adopted here is **competition ranking**: if multiple nodes share the same rank, the next rank is increased by the size of the group. For example, if two nodes are tied at rank 1, the following rank is 3.

This approach provides a clearer and more faithful representation of the PageRank results, especially for small example graphs where ties are expected.


In [3]:
from typing import Any, Dict, List, Tuple

def rank_with_ties(scores: Dict[Any, float], eps: float = 1e-12) -> List[Tuple[int, List[Tuple[Any, float]]]]:
    """
    Groups nodes with (nearly) equal scores into the same rank.
    Uses competition ranking: if rank 1 has 2 nodes tied, next rank becomes 3.
    """
    items = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)

    groups: List[Tuple[int, List[Tuple[Any, float]]]] = []
    rank = 1
    i = 0

    while i < len(items):
        base = items[i][1]
        g = [items[i]]
        i += 1

        while i < len(items) and abs(items[i][1] - base) <= eps:
            g.append(items[i])
            i += 1

        groups.append((rank, g))
        rank += len(g)

    return groups


## Application to a Real Dataset

The PageRank algorithm is applied to a real web graph provided as an edge-list dataset. After parsing the file, the graph contains 6013 nodes and 23876 directed edges. A significant fraction of the nodes (3189) are dangling nodes, i.e. pages with no outgoing links.

The PageRank vector is computed using the power method with damping factor $ (m = 0.15) $ and tolerance $ (10^{-12}) $. Convergence is achieved after 138 iterations, with a final $( \ell^1)$-difference below the prescribed tolerance. The resulting PageRank vector is properly normalized, with the sum of all scores equal to 1.

The top-ranked nodes according to their PageRank scores are reported below. These values indicate the most influential nodes in the graph under the PageRank model. Despite the large number of dangling nodes, the algorithm remains stable and converges reliably due to the uniform redistribution of dangling mass at each iteration.

Overall, the results confirm that the implementation scales to larger graphs and preserves the theoretical properties of the PageRank algorithm, including convergence and probabilistic interpretation.


In [4]:
if file_path:
    # Load dataset (edge list). Convention: u -> v means "u links to v"
    adj_dataset = load_edge_list_as_adj(file_path, flip_direction=False)

    num_nodes = len(adj_dataset)
    num_edges = sum(len(v) for v in adj_dataset.values())
    num_dangling = sum(1 for u, outs in adj_dataset.items() if len(outs) == 0)

    print("Dataset loaded from:", file_path)
    print("Nodes:", num_nodes)
    print("Edges:", num_edges)
    print("Dangling:", num_dangling)

    scores_dataset, it_dataset, last_diff_dataset = pagerank_power(
        adj_dataset, m=0.15, tol=1e-12, max_iter=2000
    )

    print("Iterations:", it_dataset)
    print("Last diff (L1):", last_diff_dataset)
    print("Sum scores:", sum(scores_dataset.values()))

    # Simple Top 10 (no tie handling)
    top10 = sorted(scores_dataset.items(), key=lambda kv: kv[1], reverse=True)[:10]
    print("Top pages by PageRank (dataset):")
    for i, (node, score) in enumerate(top10, 1):
        print(f"{i:2d}. node {node} (score={score:.6e})")

else:
    print("Dataset file 'hollins.dat' not found; skipping dataset analysis.")


Dataset loaded from: hollins.dat
Nodes: 6013
Edges: 23876
Dangling: 3189
Iterations: 138
Last diff (L1): 9.723798914500204e-13
Sum scores: 1.0
Top pages by PageRank (dataset):
 1. node 2 (score=1.987463e-02)
 2. node 37 (score=9.285693e-03)
 3. node 38 (score=8.608607e-03)
 4. node 61 (score=8.063358e-03)
 5. node 52 (score=8.024900e-03)
 6. node 43 (score=7.163157e-03)
 7. node 425 (score=6.581415e-03)
 8. node 27 (score=5.987971e-03)
 9. node 28 (score=5.570580e-03)
10. node 4023 (score=4.451544e-03)


## Validation on the Example Graphs

The PageRank algorithm is first validated on the small example graphs presented in Figures 2.1 and 2.2 of the reference paper. These cases allow a direct comparison with the theoretical results and provide a consistency check for the implementation.

### Figure 2.1

For the graph in Figure 2.1, the PageRank vector computed with damping factor \(m = 0.15\) is
$$
x \approx (0.368,\; 0.142,\; 0.288,\; 0.202)
$$
The algorithm converges in 36 iterations.  
The resulting ranking of the pages is:
$$
1 > 3 > 4 > 2
$$

This ordering reflects the link structure of the graph: page 1 receives a strong contribution from page 3, which links exclusively to it, increasing its overall importance.

---

### Figure 2.2

For the graph in Figure 2.2, which consists of two disconnected subwebs and an additional page, the computed PageRank vector is
$$
x = (0.2,\; 0.2,\; 0.285,\; 0.285,\; 0.03)
$$
Convergence is achieved after only 2 iterations.

Pages 1 and 2 have identical scores, as do pages 3 and 4, reflecting the symmetry of the two subwebs. Page 5 receives the lowest score, since it contributes outgoing links but receives no incoming links.  
The introduction of the damping factor ensures a unique PageRank vector despite the disconnected structure of the graph.


In [5]:
# Compute PageRank on Figure 2.1
a1 = build_matrix_fig21()
r1, it1 = pagerank(a1, m=0.15)
print("Figure 2.1 PageRank (m=0.15):", r1)
print("Iterations:", it1)
print("Ranking:", np.argsort(r1)[::-1] + 1)

# Compute PageRank on Figure 2.2
a2 = build_matrix_fig22()
r2, it2 = pagerank(a2, m=0.15)
print("Figure 2.2 PageRank (m=0.15):", r2)
print("Iterations:", it2)
print("Ranking:", np.argsort(r2)[::-1] + 1)


Figure 2.1 PageRank (m=0.15): [0.36815068 0.14180936 0.28796163 0.20207834]
Iterations: 36
Ranking: [1 3 4 2]
Figure 2.2 PageRank (m=0.15): [0.2   0.2   0.285 0.285 0.03 ]
Iterations: 2
Ranking: [4 3 2 1 5]


## Exercise 11: PageRank on a Modified Web Graph

In Exercise 11, the web graph of Figure 2.1 is modified by adding a fifth page and altering the link structure accordingly. The corresponding link matrix is constructed as described in the exercise, and the PageRank algorithm is applied with damping factor $m = 0.15$.

The computed PageRank vector is

$$
x \approx (0.237,\; 0.097,\; 0.349,\; 0.138,\; 0.178)
$$

and convergence is achieved after 57 iterations.  
The resulting ranking of the pages is

\[
3 > 1 > 5 > 4 > 2.
\]

The introduction of the new page and the modified links significantly changes the importance distribution, particularly increasing the score of page 3, which receives additional influence through the updated structure.

---

## Exercise 14: Convergence Analysis

Exercise 14 focuses on the convergence behavior of the PageRank iteration. Starting from a non-uniform initial vector $x_0$, the sequence

$$
x^{(k)} = M^{k} x_0
$$

is compared to the PageRank vector $q$ obtained in Exercise 11.

The $\ell^{1}$-norm of the error $\| M^{k} x_0 - q \|_{1}$ is evaluated for selected values of $k$. The results show a rapid decrease of the error, confirming exponential convergence of the power method.

In addition, a theoretical contraction bound

$$
c = \max_{j} \left| 1 - 2 \min_{i} M_{ij} \right|
$$

is computed, together with the absolute value of the second largest eigenvalue $|\lambda_{2}|$ of $M$.

The observed convergence rate is consistent with the magnitude of $|\lambda_{2}|$ and is significantly faster than the pessimistic theoretical bound $c$. This confirms both the correctness of the implementation and the effectiveness of the power method for computing PageRank.


In [6]:
# Exercise 11: compute PageRank for the modified web
a_ex11 = build_matrix_exercise11()
a_ex11_stoch = ensure_column_stochastic(a_ex11)
q, it_ex11 = pagerank(a_ex11, m=0.15)
print("Exercise 11 PageRank vector:", q)
print("Iterations:", it_ex11)
print("Ranking:", np.argsort(q)[::-1] + 1)

# Exercise 14: convergence analysis
n = a_ex11.shape[0]
m_factor = 0.15
S = np.ones((n, n)) / n
M_ex11 = (1.0 - m_factor) * a_ex11_stoch + m_factor * S

# Initial vector x0
t = np.array([0.24, 0.31, 0.08, 0.18, 0.19])
x0 = t / t.sum()

ks = [1, 5, 10, 50]
results = {}
vec = x0.copy()
prev_err = np.abs(x0 - q).sum()

for k in range(1, max(ks) + 1):
    vec = M_ex11 @ vec
    vec = vec / vec.sum()
    err = np.abs(vec - q).sum()
    if k in ks:
        ratio = err / prev_err if prev_err > 0 else None
        results[k] = (err, ratio)
    prev_err = err

mins_per_col = M_ex11.min(axis=0)
c_bound = float(np.max(np.abs(1.0 - 2.0 * mins_per_col)))
eigvals = np.linalg.eigvals(M_ex11)
abs_eigs = np.sort(np.abs(eigvals))[::-1]
lambda2_abs = float(abs_eigs[1])

print("\nExercise 14 results:")
print(f"{'k':>4} | {'||M^k x0 - q||_1':>20} | {'ratio':>12}")
print("-" * 42)
for k in ks:
    err, ratio = results[k]
    ratio_str = '---' if ratio is None else f"{ratio:.6f}"
    print(f"{k:4d} | {err:20.10e} | {ratio_str:>12}")
print(f"\nTheoretical contraction bound c = {c_bound:.6f}")
print(f"|lambda_2| = {lambda2_abs:.6f}")

Exercise 11 PageRank vector: [0.23714058 0.09718983 0.34889409 0.13849551 0.17827999]
Iterations: 57
Ranking: [3 1 5 4 2]

Exercise 14 results:
   k |     ||M^k x0 - q||_1 |        ratio
------------------------------------------
   1 |     4.2184113753e-01 |     0.784400
   5 |     4.9672424898e-02 |     0.562539
  10 |     4.2036925402e-03 |     0.614189
  50 |     1.2052220333e-11 |     0.632505

Theoretical contraction bound c = 0.940000
|lambda_2| = 0.611269


## Conclusion

In this project, the PageRank algorithm was implemented and analyzed from a linear algebra perspective. Link matrices were constructed for the small web graphs in Figures 2.1 and 2.2, and the corresponding PageRank vectors were computed using the power method. The results are consistent with the theoretical behavior discussed in the reference paper.

Exercise 11 demonstrated how modifying the link structure by adding an additional page and new connections can significantly affect the PageRank distribution and the resulting ranking. Exercise 14 provided a numerical study of the convergence of the power method, illustrating its rapid convergence and its dependence on the magnitude of the second largest eigenvalue of the Google matrix.

Finally, the implementation was extended to handle real-world graphs represented as edge lists. A dataset loader was included to construct the adjacency structure and handle dangling nodes appropriately. When a dataset file such as `hollins.dat` is available, the PageRank algorithm can be applied efficiently, confirming both the scalability and robustness of the implementation.


In [8]:
!cp '/content/drive/MyDrive/Colab Notebooks/pagerank.ipynb' /content/pagerank/

cp: cannot stat '/content/drive/MyDrive/Colab Notebooks/pagerank.ipynb': No such file or directory


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
